<a href="https://colab.research.google.com/github/podo47/Sequential-Pattern-Mining-AprioriAll/blob/main/Sequential_Pattern_Mining_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequential Pattern Mining : AprioriAll



## Import library

In [1]:
from itertools import combinations
import csv
from collections import Counter
import pandas as pd

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Calculate support

In [3]:
def supports(item,trans):
  # Count the frequency of each item
  item_freq = dict()
  for i in item:
    if type(i) is tuple:
      set_i = set(i)
    else:
      set_i = {i}
    for j in trans:
      for k in j:
        if set_i.issubset(k):
          if i in item_freq:
            item_freq[i] += 1
          else:
            item_freq[i] = 1
  return item_freq

## Litemset Phase

In [4]:
def Litemset(trans_new,min_supp):
  item = set()
  candidate = list()
  count = []
  for i in trans_new:
    for j in i:
      for k in j:
        count.append(k)
  count = Counter(count)
  min_supp = min_supp*len(trans_new)
  candidate.append({j[0]:j[1] for j in count.items() if j[1]>min_supp})
  for i in count.items():
    if i[1]>min_supp:
      item.add(i[0])
  for i in trans_new:
    for j in i:
      for k in j.copy():
        if k not in item:
          j.remove(k)
  max_trans_size = len(item)
  n = 2
  i = 1
  while n < max_trans_size:
    if len(item)<n:
      break
    item_combin = combinations(item,n)
    supp_combin = supports(item_combin,trans_new)
    if list({j[0]:j[1] for j in supp_combin.items() if j[1]>min_supp}) != []:
        candidate.append({j[0]:j[1] for j in supp_combin.items() if j[1]>min_supp})
    if len(candidate)-1 < i:
      break
    item_test = set()
    for j in candidate[i].keys():
      for k in j:
        item_test.add(k)
    item = item_test
    n+=1
    i+=1


  return candidate

## Transformation Phase

In [5]:
def Transformation(trans_new,candidate): 
  map = 1
  map_dict = {}
  for i in candidate:
    for j in i.keys():
      if type(j) is tuple:
        map_dict[map] = set(j)
      else:
        map_dict[map] = {j}
      map += 1 
  trans_map = []
  list_of_key = list(map_dict.keys())
  list_of_value = list(map_dict.values())

  for i in range(len(trans_new)):
    seq_map = []
    seq = trans_new[i]
    for j in range(len(seq)):
      ele_set = set()
      element = seq[j]
      if len(element)>0:
        for k in map_dict.values():
          if k.issubset(element):
            if len(k)>1:
              for m in k:
                m = {m}
                pos = list_of_value.index(m)
                val = list_of_key[pos]
                ele_set.add(val)
              pos = list_of_value.index(k)
              val = list_of_key[pos]
              ele_set.add(val)
            else:
              pos = list_of_value.index(k)
              val = list_of_key[pos]
              ele_set.add(val)
      seq_map.append(ele_set)
    trans_map.append(seq_map)

  for i in trans_map:
    for j in i.copy():
      if len(j)<1:
        i.remove(j)

  trans_map = [ i for i in trans_map if i !=[]]

  return trans_map,map_dict

## Maximal phase

In [43]:
def Maximal(sequence,map_dict):
  len_seq = len(sequence)
  for i in range(len_seq-1,0,-1):
    list_of_key = list(sequence[i].keys())
    list2_of_key = list(sequence[i-1].keys())
    for j in list_of_key:
      for k in list2_of_key:
        if type(k) is tuple:
          if set(k).issubset(set(j)):
            sequence[i-1][k] = []
        else:
          if {k}.issubset(set(j)):
            sequence[i-1][k] = []
  del sequence[0]
  sequence = [ i for i in sequence if i !={}]

  result_dict = {}
  for i in range(len(sequence)):
    for j in sequence[i].items():
      result = tuple()
      key = j[0]
      for k in key:
        result += (map_dict[k],)
      if j[1] !=[]:
        result_dict[str(result)] = j[1]


  return result_dict

## Answer form

In [52]:
def ans(result_dict):
  list_of_key = list(result_dict.keys())
  list_of_val = list(result_dict.values())
  with open('/content/drive/MyDrive/Data_Mining/Sequential/output.csv', 'w', newline='') as csvfile:
    fieldnames = [' pattern','→','Support']
    csvfile = csv.writer(csvfile)
    csvfile.writerow(fieldnames)
    for i in list_of_key:
      csvfile.writerow([i,'→',result_dict[i]])


# Run

## Input file preprocessing

### Read input

In [59]:
path = '/content/drive/MyDrive/Data_Mining/Sequential/seqdata.dat.txt'

### Modify input format and store in list

In [60]:
trans = []
with open(path) as ip:
  for line in ip:
      sp = line.split()
      sp = [int(i) for i in sp]
      trans.append(sp)

In [61]:
for i in range(len(trans)):
  del trans[i][0]

In [62]:
trans_new = []
for i in range(len(trans)):
  cus = trans[i]
  half_cus_len = int(len(cus)/2)
  id_val = 0
  cus_m = []
  for j in range(half_cus_len):
    id = 2*j
    if cus[id] != id_val:
      if id == 0:
        item_set = {cus[id+1]}
        id_val = cus[id]
      else:
        cus_m.append(item_set)
        item_set = {cus[id+1]}
        id_val = cus[id]
    else:
      item_set.add(cus[id+1])
  cus_m.append(item_set)
  trans_new.append(cus_m)

## Main : Sequential Pattern Mining Algorithm

In [53]:
def Sequential():
  min_supp = float(input("Please enter min support : ")) # User-defined : 0 < input < 1 - Please enter min support : 0.01
  litem = Litemset(trans_new,min_supp)
  transform,map_diction = Transformation(trans_new,litem)
  sequence = Litemset(transform,min_supp*2)
  maxi = Maximal(sequence,map_diction)
  answer = ans(maxi)
  return maxi

# Output

In [63]:
test = Sequential() #Please enter min support : 0.01
df = pd.read_csv ('/content/drive/MyDrive/Data_Mining/Sequential/output.csv')
df

Please enter min support : 0.01


,pattern,→,Support
0,"({3488}, {1566}, {3488, 1566})",→,326
1,"({5447}, {5493}, {5493, 5447})",→,406
2,"({7160}, {6549}, {7160, 6549})",→,240
3,"({552}, {4811}, {552, 4811})",→,348
4,"({7125}, {2240, 7125}, {2240})",→,318
5,"({8208, 7775}, {7775}, {8208})",→,255
6,"({1946, 4683}, {1946}, {4683})",→,262
7,"({8969, 2652}, {8969}, {2652})",→,349
8,"({2722, 1503}, {2722}, {1503})",→,264
9,"({6312, 1807}, {6312}, {1807})",→,297
